# 加入出发地当天天气信息

In [1]:
import pandas as pd

In [2]:
# 创建出发地天气id字典
airport = pd.read_csv('./dataset/temp_airport.csv', encoding= 'gbk')
# 读取airport.csv文件为字典
weather_dict = {}
for i in range(len(airport)):
    weather_dict[airport['机场编码'][i]] = airport['网站ID'][i]
print(weather_dict)

{'JDZ': 58527, 'PVG': 58362, 'TGO': 54135, 'XMN': 59134, 'URC': 51463, 'TYN': 53772, 'SZX': 59493, 'SYX': 59948, 'TAO': 54857, 'TNA': 54823, 'SJW': 53698, 'SHE': 54342, 'SHA': 58362, 'PEK': 54511, 'NKG': 58238, 'KWL': 57957, 'KMG': 56778, 'HRB': 50953, 'HET': 53463, 'HGH': 58457, 'FOC': 58847, 'CTU': 57516, 'CKG': 57516, 'HFE': 58321, 'CGO': 57083, 'TLQ': 51573, 'WUH': 57494}


# 进入网站爬取数据


In [3]:
#将id变为str
from Write import write
for city in weather_dict:
    id = str(weather_dict[city])
    write(city, id)
    print(city, '(', id, ')已写入csv文件')

http://www.meteomanz.com/sy2?l=1&cou=2250&ind=58527&d1=01&m1=01&y1=2015&d2=28&m2=01&y2=2015
写入完成
http://www.meteomanz.com/sy2?l=1&cou=2250&ind=58527&d1=01&m1=02&y1=2015&d2=28&m2=02&y2=2015
写入完成
http://www.meteomanz.com/sy2?l=1&cou=2250&ind=58527&d1=01&m1=03&y1=2015&d2=28&m2=03&y2=2015
写入完成
http://www.meteomanz.com/sy2?l=1&cou=2250&ind=58527&d1=01&m1=04&y1=2015&d2=28&m2=04&y2=2015
写入完成
http://www.meteomanz.com/sy2?l=1&cou=2250&ind=58527&d1=01&m1=05&y1=2015&d2=28&m2=05&y2=2015
写入完成
http://www.meteomanz.com/sy2?l=1&cou=2250&ind=58527&d1=01&m1=06&y1=2015&d2=28&m2=06&y2=2015
写入完成
http://www.meteomanz.com/sy2?l=1&cou=2250&ind=58527&d1=01&m1=07&y1=2015&d2=28&m2=07&y2=2015
写入完成
http://www.meteomanz.com/sy2?l=1&cou=2250&ind=58527&d1=01&m1=08&y1=2015&d2=28&m2=08&y2=2015
写入完成
http://www.meteomanz.com/sy2?l=1&cou=2250&ind=58527&d1=01&m1=09&y1=2015&d2=28&m2=09&y2=2015
写入完成
http://www.meteomanz.com/sy2?l=1&cou=2250&ind=58527&d1=01&m1=10&y1=2015&d2=28&m2=10&y2=2015
写入完成
http://www.meteomanz.com/sy2?l

# 将气象数据合并至flight_reduced_unique_calculated.csv中并作为数据集

In [12]:
# 读取flight_reduced_unique_calculated.csv文件
dataSet = pd.read_csv('./dataset/flight_reduced_unique_calculated.csv', encoding='utf-8')

# 创建出发地天气id字典
airport = pd.read_csv('./dataset/airport.csv', encoding= 'gbk')
# 读取airport.csv文件为列表
airport_list = []
for i in range(len(airport)):
    airport_list.append(airport['机场编码'][i])

# 删除无气象数据的行
dataSet = dataSet[dataSet['出发机场'].isin(airport_list)]
dataSet = dataSet[dataSet['到达机场'].isin(airport_list)]

# 读取各机场的天气csv文件
airport_weather = {}
for i in range(len(airport_list)):
    airport_weather[airport_list[i]] = pd.read_csv('./dataset/weather/' + airport_list[i] + '.csv', encoding='utf-8')

drop_list = []

# 添加天气数据到原始数据集
dataSet['平均温度'] = float(0)
dataSet['最高温度'] = float(0)
dataSet['最低温度'] = float(0)
dataSet['降水量'] = float(0)
dataSet['气压'] = float(0)
dataSet['风向'] = float(0)
dataSet['风速'] = float(0)
for i in range(len(dataSet)):
    # 通过机场编码和日期找到对应的天气数据
    airport_code = dataSet['出发机场'].values[i]
    date = dataSet['计划出发日期'].values[i]
    # 将日期变为字符串
    date = str(date).split('-')[2] + '/' + str(date).split('-')[1] + '/' + str(date).split('-')[0]
    # 找到对应日期的天气数据
    for j in range(len(airport_weather[airport_code])):
        if airport_weather[airport_code]['Time'][j] == date:
            dataSet['平均温度'].values[i] = airport_weather[airport_code]['Ave_t'].values[j]
            dataSet['最高温度'].values[i] = airport_weather[airport_code]['Max_t'].values[j]
            dataSet['最低温度'].values[i] = airport_weather[airport_code]['Min_t'].values[j]
            dataSet['降水量'].values[i] = airport_weather[airport_code]['Prec'].values[j]
            dataSet['气压'].values[i] = airport_weather[airport_code]['SLpress'].values[j]
            dataSet['风向'].values[i] = airport_weather[airport_code]['Winddir'].values[j]
            dataSet['风速'].values[i] = airport_weather[airport_code]['Windsp'].values[j]
            break

# 保存数据集
dataSet.to_csv('./dataset/flight_final.csv', encoding='utf-8',index = False)


# 删除没有气象数据的行

In [ ]:
# 读取flight_final.csv文件
dataSet = pd.read_csv('./dataset/flight_final.csv', encoding='utf-8')

# 删除平均温度，最高温度，最低温度，降水量，气压，风向，风速为0的行
dataSet = dataSet[dataSet['平均温度'] != 0]

# 保存数据集
dataSet.to_csv('./dataset/flight_final_no_zero.csv', encoding='utf-8',index = False)